<a href="https://colab.research.google.com/github/marabian/toxic-comment-classification/blob/master/toxic-comment-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Toxic Comment Classification
***

[Link to Kaggle challenge/dataset](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/)


In [1]:
import os
import random as rnd

# data analysis and wrangling
import pandas as pd
import numpy as np


# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# to mount google drive for colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# check gpu support
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
# check if gpu working
# tf.test.gpu_device_name()

In [ ]:
# change notebook theme
#!jt -t monokai -T -N -kl

## Workflow


* Defining the problem and assembling a dataset<br>


* Choosing a measure of success

* Deciding on an evaluation protocol


* Preparing your data for ML algorithms


* Developing a model that does better than a baseline


* Scaling up: developing a model that overfits


* Regularizing your model and tuning your hyperparameters



## Defining the problem and assembling a dataset
***


We are challenged to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate. Our training data will consist of ~160,000 comments from Wikipedia’s talk page edits with labeled levels of toxicity. Detecting toxic comments online will hopefully help online discussion become more productive and respectful.<br>

Since we have many classes, this problem is an instance of multiclass classification; and because each data point could belong to multiple categories (in this case, levels of toxicity), this is a **multilabel, multiclass classification problem**.

We will start with a solving a subset of this problem, using **binary classification** to classify between "good" and "toxic" comments.

In [8]:
path_to_data = "/content/drive/My Drive/machine learning/notebooks/data/jigsaw-toxic-comment-classification-challenge/"

In [15]:
train_df = pd.read_csv(path_to_data + "train.csv")
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_df



,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [20]:
train_df[train_df[labels].eq(0).all(1)][['id', 'comment_text', 'toxic']]

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [23]:
train_df[train_df['toxic'] == 1][['id', 'comment_text', 'toxic']]

,id,comment_text,toxic
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1


In [21]:
pd.concat([train_df[train_df[labels].eq(0).all(1)], bad_df]).reset_index(drop=True)[['id', 'comment_text', 'toxic']]

NameError: ignored

In [25]:
train_df[0:10]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0
